In [ ]:
# install gensim
import sys
!{sys.executable} -m pip install gensim --user

In [ ]:
# install xz related to lzma
%cd Reddit
!curl -L -O http://tukaani.org/xz/xz-5.0.4.tar.gz
!tar -zxvf xz-5.0.4.tar.gz
%cd xz-5.0.4
!./configure --prefix=$HOME
!make
!make check
!make install

In [ ]:
# install lzma to uncompress xz file
import sys
!{sys.executable} -m pip install backports.lzma --user

In [ ]:
# word2vec (all score)

import json
import gensim
import multiprocessing
import bz2
import math
import os
import sys
import datetime
import time
from backports import lzma

%cd Reddit

# hyper parameters of word2vec
eb_size = 100
w2v_win = 5
mincount = 100
epoch_size = 10

path = "model_save_{0}_{1}_{2}_{3}".format(eb_size,w2v_win,mincount,epoch_size)

# parameters to continue training
ini = 1 # 0 if continue to train, 1 if initiate to train
#year = 2017
#month = 11
#day_m = 30
#model = gensim.models.Word2Vec.load(path + "/w2v_trained_reddit_{0}_{1}_{2}_{3}_{4}_{5}_{6}".format(year,int(month),day_m,eb_size,w2v_win,mincount,epoch_size))

if os.path.isdir(path) == False:
    os.mkdir(path)
     
sys.stdout = open("model_log_{0}_{1}_{2}_{3}.txt".format(eb_size,w2v_win,mincount,epoch_size), "a")

print("word2vec training")
print(datetime.datetime.now())
print("embedding_size: {0}".format(eb_size))
print("w2v_window_size: {0}".format(w2v_win))
print("w2v_min_count: {0}".format(mincount))
print("w2v_epoch_size: {0}".format(epoch_size))
sys.stdout.flush()

# there are few lines that are not in json format
def is_json(line):
    try:
        json_object = json.loads(line)
    except ValueError:
        return False
    return True


for i in range(2011,2018):
    for ii in range(1,13):
        year = i
        if ii<10:
            month = "0{0}".format(ii)
        else:
            month = ii
            
        print("start: read file from bz2: {0}_{1}".format(year,month))
        print(datetime.datetime.now())
        sys.stdout.flush()
        
        if os.path.isfile("RS_{0}-{1}.bz2".format(year,month)):
                data_file = bz2.BZ2File("RS_{0}-{1}.bz2".format(year,month)).readlines()
        elif os.path.isfile("RS_{0}-{1}.xz".format(year,month)):
                data_file = lzma.open("RS_{0}-{1}.xz".format(year,month)).readlines()
   
        data_time = []
        data_title = []
        data_time_sort = []
        data_title_sort = []
        for line in data_file:
            if is_json(line) is True:
                line_load = json.loads(line)
                data_time.append(int(line_load["created_utc"]))
                data_title.append(gensim.utils.simple_preprocess(line_load["title"]))
              
        data_time_sort = sorted(data_time)
        data_title_sort = [x for _,x in sorted(zip(data_time,data_title))]
        
        data_file.clear()
        data_time.clear()
        data_title.clear()
        
        print("end: read file from bz2: {0}_{1}".format(year,month))
        print(datetime.datetime.now())
        sys.stdout.flush()
        
        for ind, ut in enumerate(data_time_sort):
            ut_to_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ut))
            day_data = int(ut_to_time[8:10])
            if ind == 0:
                day_data_temp = day_data
                temp = []
            
            if day_data == day_data_temp: # cumulate data
                temp.append(data_title_sort[ind])   
            if day_data != day_data_temp or ind == len(data_time_sort) - 1: # update day & traning previous data(temp)
                print([year,int(month),day_data_temp,datetime.datetime.now()])
                sys.stdout.flush()
                # training previous updated data
                if ini == 1: # initial word2vec training
                    model = gensim.models.Word2Vec(temp, size=eb_size, window=w2v_win, min_count=mincount,workers=2*multiprocessing.cpu_count())
                    ini = 0
                else: # online word2vec training
                    model.build_vocab(temp, update=True)
                model.train(temp,total_examples=len(temp),epochs=epoch_size)
                model.save(path + "/w2v_trained_reddit_{0}_{1}_{2}_{3}_{4}_{5}_{6}".format(year,int(month),day_data_temp,eb_size,w2v_win,mincount,epoch_size))
                
                temp = []
                temp.append(data_title_sort[ind])
            day_data_temp = day_data


In [ ]:
# word2vec (only score larger than score threshold)
# remove stop words

import json
import gensim
import multiprocessing
import bz2
import math
import os
import sys
import datetime
import time
from backports import lzma
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.corpora import Dictionary

%cd Reddit

# hyper parameters of word2vec
eb_size = 50
w2v_win = 5
mincount = 100
epoch_size = 10

score_threshold = 10

path = "model_save_{0}_{1}_{2}_{3}_recommend".format(eb_size,w2v_win,mincount,epoch_size)

stop_words = set(stopwords.words("english"))

# parameters to continue training
ini = 1 # 0 if continue to train, 1 if initiate to train
#year = 2017
#month = 9
#day_m = 30
#model = gensim.models.Word2Vec.load(path + "/w2v_trained_reddit_{0}_{1}_{2}_{3}_{4}_{5}_{6}_recommend".format(year,int(month),day_m,eb_size,w2v_win,mincount,epoch_size))

if os.path.isdir(path) == False:
    os.mkdir(path)
     
sys.stdout = open("model_log_{0}_{1}_{2}_{3}_recommend.txt".format(eb_size,w2v_win,mincount,epoch_size), "a")

print("word2vec training")
print(datetime.datetime.now())
print("embedding_size: {0}".format(eb_size))
print("w2v_window_size: {0}".format(w2v_win))
print("w2v_min_count: {0}".format(mincount))
print("w2v_epoch_size: {0}".format(epoch_size))
sys.stdout.flush()

# there are few lines that are not in json format
def is_json(line):
    try:
        json_object = json.loads(line)
    except ValueError:
        return False
    return True


for i in range(2011,2018):
    for ii in range(1,13):
        year = i
        if ii<10:
            month = "0{0}".format(ii)
        else:
            month = ii
            
        print("start: read file from bz2: {0}_{1}".format(year,month))
        print(datetime.datetime.now())
        sys.stdout.flush()
        
        if os.path.isfile("RS_{0}-{1}.bz2".format(year,month)):
                data_file = bz2.BZ2File("RS_{0}-{1}.bz2".format(year,month)).readlines()
        elif os.path.isfile("RS_{0}-{1}.xz".format(year,month)):
                data_file = lzma.open("RS_{0}-{1}.xz".format(year,month)).readlines()
   
        data_time = []
        data_title = []
        data_time_sort = []
        data_title_sort = []
        for line in data_file:
            if is_json(line) is True:
                line_load = json.loads(line)
                if line_load["score"] is not None:
                    if int(line_load["score"]) > score_threshold:
                        data_time.append(int(line_load["created_utc"]))
                        temp_title = gensim.utils.simple_preprocess(line_load["title"])
                        data_title.append([temp_title for temp_title in temp_title if temp_title not in stop_words])
                        #data_title.append(gensim.utils.simple_preprocess(line_load["title"]))
              
        data_time_sort = sorted(data_time)
        data_title_sort = [x for _,x in sorted(zip(data_time,data_title))]
        
        data_file.clear()
        data_time.clear()
        data_title.clear()
        
        print("end: read file from bz2: {0}_{1}".format(year,month))
        print(datetime.datetime.now())
        sys.stdout.flush()
        
        for ind, ut in enumerate(data_time_sort):
            ut_to_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ut))
            day_data = int(ut_to_time[8:10])
            if ind == 0:
                day_data_temp = day_data
                temp = []
            
            if day_data == day_data_temp: # cumulate data
                temp.append(data_title_sort[ind])   
            if day_data != day_data_temp or ind == len(data_time_sort) - 1: # update day & traning previous data(temp)
                print([year,int(month), day_data_temp, datetime.datetime.now()])
                sys.stdout.flush()
                
                '''
                # gensim filter_extreme
                temp_dic = Dictionary(temp)
                #temp_dic.filter_extremes(no_below=1, no_above=0.2, keep_n=100000, keep_tokens=None)
                temp_dic.filter_extremes(no_below=1, no_above=0.3)
                
                temp_list = []
                for fi in range(len(temp_dic)):
                    temp_list.append(temp_dic[fi])

                temp_after_filter = []
                for fi in temp:
                    temp_after_filter.append([fi for fi in fi if fi in set(temp_list)])
                ''' 
                
                # training previous updated data
                if ini == 1: # initial word2vec training
                    model = gensim.models.Word2Vec(temp, size=eb_size, window=w2v_win, min_count=mincount,workers=2*multiprocessing.cpu_count())
                    ini = 0
                else: # online word2vec training
                    model.build_vocab(temp, update=True)
                model.train(temp,total_examples=len(temp),epochs=epoch_size)
                model.save(path + "/w2v_trained_reddit_{0}_{1}_{2}_{3}_{4}_{5}_{6}_recommend".format(year,int(month),day_data_temp,eb_size,w2v_win,mincount,epoch_size))
                # del temp_after_filter, temp_dic
                
                temp = []
                temp.append(data_title_sort[ind])
            day_data_temp = day_data
